# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol= 'AAPL'
api_url= f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data= requests.get(api_url).json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
col = ['Ticker','Stock Price','Market Cap', 'Purchase Amount']
final_df = pd.DataFrame(columns=col)

In [7]:
final_df.append(
    pd.Series(
        [symbol,price,market_cap,'N/A'],
        index=col
        ),
    ignore_index=True
    )

,Ticker,Stock Price,Market Cap,Purchase Amount
0,AAPL,139.26,2305692092982,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_df = pd.DataFrame(columns=col)
for stock in stocks['Ticker']:
    api_url= f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data= requests.get(api_url).json()

    final_df = final_df.append(
    pd.Series(
        [symbol,
        data['latestPrice'],
        data['marketCap'],
        'N/A'],
        index=col
        ),
    ignore_index=True
    )



In [9]:
final_df

,Ticker,Stock Price,Market Cap,Purchase Amount
0,AAPL,122.18,36594190548,N/A
1,AAPL,16.09,9907067882,N/A
2,AAPL,159.39,10979758230,N/A
3,AAPL,134.53,2265936029614,N/A
4,AAPL,111.49,198505517312,N/A
...,...,...,...,...
500,AAPL,109.32,33433052531,N/A
501,AAPL,155.06,33105568367,N/A
502,AAPL,386.67,21059178318,N/A
503,AAPL,44.45,7225752966,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst, n):
    """Yields successive n-sized chunks from lst."""
    for i in range(0,len(lst), n):
        yield lst[i:i+n]

In [11]:
symbol_batches = list(chunks(stocks['Ticker'],100))
symbol_stings = []
for i in range(0, len(symbol_batches)):
    symbol_stings.append(','.join(symbol_batches[i]))

In [12]:
final_df = pd.DataFrame(columns=col)

for symbol_sting in symbol_stings:
    batch_api_call_url= f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_sting}&types=quote&token={IEX_CLOUD_API_TOKEN}'

    data= requests.get(batch_api_call_url).json()

    for symbol in symbol_sting.split(','):
        final_df = final_df.append(
        pd.Series(
            [symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'],
            index=col
            ),
        ignore_index=True
        )

In [13]:
final_df

,Ticker,Stock Price,Market Cap,Purchase Amount
0,A,119.08,38010955677,N/A
1,AAL,16.25,9791261515,N/A
2,AAP,160.27,11098924235,N/A
3,AAPL,137.24,2323374318049,N/A
4,ABBV,109.59,196595613389,N/A
...,...,...,...,...
500,YUM,112.95,32811239424,N/A
501,ZBH,159.54,32660255798,N/A
502,ZBRA,396.44,20635924722,N/A
503,ZION,43.56,7332568008,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [36]:

while True:
    try:
        portfolio_size = float(input('Enter the value of your portfolio:'))

    except ValueError:
        print ("Oops!  That was not a valid number.  Try again...")
        
    else:
        break

portfolio_size

1000000.0

In [37]:
position_size = portfolio_size/len(final_df.index)
for i in range (0, len(final_df.index)):
    final_df.loc[i,'Purchase Amount'] = math.floor(position_size/final_df.loc[i,'Stock Price'])

In [38]:
final_df

,Ticker,Stock Price,Market Cap,Purchase Amount
0,A,119.08,38010955677,16
1,AAL,16.25,9791261515,121
2,AAP,160.27,11098924235,12
3,AAPL,137.24,2323374318049,14
4,ABBV,109.59,196595613389,18
...,...,...,...,...
500,YUM,112.95,32811239424,17
501,ZBH,159.54,32660255798,12
502,ZBRA,396.44,20635924722,4
503,ZION,43.56,7332568008,45


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [72]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine= 'xlsxwriter')
final_df.to_excel(writer, sheet_name= 'Recommended Trades', index= False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [73]:
background_color = '#0a0a23'
font_color = 'ffffff'

string_format = writer.book.add_format(
 {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
    }
)

dollar_format = writer.book.add_format(
 {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
    }
)


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [74]:
writer.sheets['Recommended Trades'].set_column('A:A', 10, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 10, string_format)
writer.sheets['Recommended Trades'].set_column('C:C', 10, string_format)
writer.sheets['Recommended Trades'].set_column('D:D', 10, string_format)
writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [75]:
column_formats = {
    'A':['Ticker',string_format],
    'B':['Stock Price',dollar_format],
    'C':['Market Cap',dollar_format],
    'D':['Purchase Amount',integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 10, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0],column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [76]:
writer.save()